In [1]:
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras.layers import Dense

#from sklearn.neural_network import MLPRegressor
#from sklearn.metrics import mean_squared_error

In [2]:
'''MLPRegressor(hidden_layer_sizes=(900,7), activation='relu',
             solver='adam',alpha=0.001, 
             batch_size=13, learning_rate='constant',learning_rate_init=0.001, 
             max_iter=1000, shuffle=True,random_state=5, tol=0.001, 
             early_stopping=True,
             verbose=True,
             validation_fraction=0.2
             )'''

path_to_data = ('ngp.csv','timesereis_8_2.csv')
target = 'price'

cur_hidden_layer_sizes = (900,7)
cur_act_fun = 'relu'
cur_solver = 'adam'
cur_alpha = 0.001
cur_batch_size = 13
cur_learning_rate = 'constant'
cur_learning_rate_init = 0.001
cur_max_iter = 1000
cur_shuffle = True
cur_random_state = 5
cur_tol = 0.001
cur_early_stopping = True
cur_verbose = True
cur_validation_fraction = 0.2

In [3]:
def get_df(path, target_name):
    print("///////// get_df /////////")
    file = pd.read_csv(path)
    dframe = pd.DataFrame(list(reversed(file[target_name])))
    print(dframe)
    print("///////// get_df_end /////////")
    return dframe

In [4]:
def get_cols_and_names(df):
    input_t = 4
    output_t = 1
    
    cols = list()
    names = list()
    
    print("///////// get_cols_and_names /////////")
    for i in range(input_t,0, -1):
        cols.append(df.shift(i))
        #cols.append(np.array(df.shift(i))
        print(f"df.shift({i}).shape: ", df.shift(i).shape, "type: ", type(df.shift(i)))
        names.append(i)
    print("///////// Next loop /////////")
    for i in range(0,output_t):
        #cols.append(np.array(df.shift(-i))
        cols.append(df.shift(-i))                    
        print(f"df.shift({-i}).shape: ", df.shift(-i).shape)
        names.append(i+input_t+1)
        
    #print("\ncols.shape: ", np.array(cols).shape)
    #print("names: ", names)
    
    #print("len(cols)", len(cols))
    #print("len(cols[0])", len(cols[0]))
    #print("len(cols[0][0])", len(cols[0][0]))
    
    #print("cols[0][0]: ", cols[0][0])
    #print("cols[0]: ", cols[0])
    
    #print("Cols: ", cols)
    
    print("///////// get_cols_and_names_end /////////")
    return cols,names

In [5]:
def split(X,Y,factor):
    X_train=X[:factor]
    Y_train=Y[:factor]
    X_test=X[factor:]
    Y_test=Y[factor:]
    return X_train,Y_train,X_test,Y_test

In [6]:
def prepare_df(cols,names):
    print("///////// prepare_df /////////")
    n_df = pd.concat(cols, axis = 1)
    n_df.columns = names
    
    print("n_df: ", n_df)
    
    n_df.dropna(inplace = True)
    
    print("n_df: ", n_df)
    
    print("///////// prepare_df_end /////////")
    return n_df

In [7]:
from IPython.display import display
def prep_data_4_1_time(cols, names, prepared_df):
    print("///////// prep_data_4_1_time /////////")
 
    X = prepared_df[[4, 3, 2, 1]]
    Y = prepared_df[5]
    X = X / X.max()
    #ones = np.ones(shape = (X.shape[0], 1))
    #X = np.append(X, ones, axis = 1)
    Y = np.asarray(Y)
    Y = Y / Y.max()

    #print("X[0], Y[0]", X[0], Y[0])
    #print("X[1], Y[1]", X[1], Y[1])
    
    factor = int(.80*X.shape[0])

    print("///////// prep_data_4_1_time_end /////////")
    return split(X ,Y , factor)

In [8]:
#prep_data_4_1_time()
#pd.DataFrame.dropna?
#np.asarray?

In [9]:
'''def prep_data_8_2_time(path):
    file = pd.read_csv(path)
    
    X = file[['0','1','2','3','4','5','6','7']]
    X = X / X.max()
    Y = file[['8','9']]
    Y = Y / Y.max()

    ones = np.ones(shape = (X.shape[0], 1))
    X = np.append(X, ones, axis=1)
    Y = np.asarray(Y)
    factor=int(.80 * X.shape[0])

    return split(X, Y, factor)'''

"def prep_data_8_2_time(path):\n    file = pd.read_csv(path)\n    \n    X = file[['0','1','2','3','4','5','6','7']]\n    X = X / X.max()\n    Y = file[['8','9']]\n    Y = Y / Y.max()\n\n    ones = np.ones(shape = (X.shape[0], 1))\n    X = np.append(X, ones, axis=1)\n    Y = np.asarray(Y)\n    factor=int(.80 * X.shape[0])\n\n    return split(X, Y, factor)"

In [10]:
'''clf=MLPRegressor(
    hidden_layer_sizes = cur_hidden_layer_sizes, 
    activation = cur_act_fun,
    solver = cur_solver,
    alpha = cur_alpha, 
    batch_size = cur_batch_size, 
    learning_rate = cur_learning_rate,
    learning_rate_init = cur_learning_rate_init, 
    max_iter = cur_max_iter, 
    shuffle = cur_shuffle,
    random_state = cur_random_state, 
    tol = cur_tol, 
    early_stopping = cur_early_stopping,
    verbose=cur_verbose,
    validation_fraction=cur_validation_fraction
    )'''

model = keras.Sequential([Dense(4, activation = 'relu'),
                   Dense(900, activation = 'relu'),
                   Dense(7, activation = "relu"),
                   Dense(1, activation = "linear")])

model.compile(loss='mean_squared_error', optimizer = keras.optimizers.Adam(0.001))

'''clf=MLPRegressor(
    hidden_layer_sizes=(900,7), # 2 скрытых слоя, первый с 900 нейронами, второй с 7 ?
    activation='relu', # функция активации скрытых слоев
    solver='adam', # метод оптимизации
    alpha=0.001, #
    batch_size=13, # количество наборов данных, подаваемых в сеть для обучения (1,1,1 -> результат 4), (1,1,2 -> результат 2). 
    # тогда batch_size = 2
    learning_rate='constant', # изменять ли шаг сходимости по мере обучения сети
    learning_rate_init=0.001, # начальный шаг  сходимости
    max_iter=1000, # макс кол-во итераций на эпоху?
    shuffle=True, # перемешивать ли  данный в каждой итерации
    random_state=5, # случайная генерация чисел происходит только 1 раз, все последующие разы она не меняется 
    # для весов и мб биасов
    tol=0.001, # минимальное изменение loss для какое-то кол-во итераций (10 по умолчанию), при котором стоит 
    # продолжать обучение
    early_stopping=True, # остановка обучения, если сеть перестала обучаться
    verbose=True, # выводить ли в консоль прогресс обучения
    validation_fraction = 0.2 # процент данных, отсекаемых для валидационной выборки
    )'''

"clf=MLPRegressor(\n    hidden_layer_sizes=(900,7), # 2 скрытых слоя, первый с 900 нейронами, второй с 7 ?\n    activation='relu', # функция активации скрытых слоев\n    solver='adam', # метод оптимизации\n    alpha=0.001, #\n    batch_size=13, # количество наборов данных, подаваемых в сеть для обучения (1,1,1 -> результат 4), (1,1,2 -> результат 2). \n    # тогда batch_size = 2\n    learning_rate='constant', # изменять ли шаг сходимости по мере обучения сети\n    learning_rate_init=0.001, # начальный шаг  сходимости\n    max_iter=1000, # макс кол-во итераций на эпоху?\n    shuffle=True, # перемешивать ли  данный в каждой итерации\n    random_state=5, # случайная генерация чисел происходит только 1 раз, все последующие разы она не меняется \n    # для весов и мб биасов\n    tol=0.001, # минимальное изменение loss для какое-то кол-во итераций (10 по умолчанию), при котором стоит \n    # продолжать обучение\n    early_stopping=True, # остановка обучения, если сеть перестала обучаться\n  

In [17]:
#MLPRegressor?
keras.Sequential.compile?

In [12]:
df = get_df(path_to_data[0], target)
cols, names = get_cols_and_names(df)
#print(df)
prepared_df = prepare_df(cols,names)
prepared_data = prep_data_4_1_time(cols,names,prepared_df)

X_train,Y_train,X_test,Y_test = prepared_data
#train the model
#clf.fit(X_train,Y_train)
#score of the model
#print("score of first model",clf.score(X_test,Y_test))

///////// get_df /////////
         0
0     3.79
1     4.19
2     2.98
3     2.91
4     2.53
...    ...
1077  2.93
1078  2.91
1079  2.96
1080  3.09
1081  2.96

[1082 rows x 1 columns]
///////// get_df_end /////////
///////// get_cols_and_names /////////
df.shift(4).shape:  (1082, 1) type:  <class 'pandas.core.frame.DataFrame'>
df.shift(3).shape:  (1082, 1) type:  <class 'pandas.core.frame.DataFrame'>
df.shift(2).shape:  (1082, 1) type:  <class 'pandas.core.frame.DataFrame'>
df.shift(1).shape:  (1082, 1) type:  <class 'pandas.core.frame.DataFrame'>
///////// Next loop /////////
df.shift(0).shape:  (1082, 1)
///////// get_cols_and_names_end /////////
///////// prepare_df /////////
n_df:           4     3     2     1     5
0      NaN   NaN   NaN   NaN  3.79
1      NaN   NaN   NaN  3.79  4.19
2      NaN   NaN  3.79  4.19  2.98
3      NaN  3.79  4.19  2.98  2.91
4     3.79  4.19  2.98  2.91  2.53
...    ...   ...   ...   ...   ...
1077  2.80  2.85  2.95  2.97  2.93
1078  2.85  2.95  2.97  2

In [13]:
history = model.fit(X_train, Y_train, batch_size = 13, epochs = 1, verbose = True, validation_split = 0.2)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

53/53 [==============================] - 0s 4ms/step - loss: 0.0358 - val_loss: 0.0084


In [14]:
#test the model
#pred = clf.predict(X_test)
#print("error of first model",mean_squared_error(Y_test,pred))
model.evaluate(X_test, Y_test)

7/7 [==============================] - 0s 571us/step - loss: 0.0125


0.012525551952421665

In [15]:
'''X_train,Y_train,X_test,Y_test=prep_data_8_2_time(path_to_data[1])
#train the model
clf.fit(X_train,Y_train)
#score of the model
print("score of second model",clf.score(X_test,Y_test));'''

'X_train,Y_train,X_test,Y_test=prep_data_8_2_time(path_to_data[1])\n#train the model\nclf.fit(X_train,Y_train)\n#score of the model\nprint("score of second model",clf.score(X_test,Y_test));'

In [16]:
'''#test the model
pred = clf.predict(X_test)
print("error of second model",mean_squared_error(Y_test,pred))'''

'#test the model\npred = clf.predict(X_test)\nprint("error of second model",mean_squared_error(Y_test,pred))'